## Hierarchical Clustering

In this notebook, you'll see how to perform hierarchical clustering on the voting records of the US Senate.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.preprocessing import LabelEncoder

from scipy.spatial.distance import pdist, squareform

The file votes.csv contains the votes of all senators from the first session of the 117th Congress and was scraped from https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_1.htm.

In [ ]:
votes = pd.read_csv('data/votes.csv').dropna()
votes.head()

In order to perform clustering, we need to have a way to measure the similarity of two senators. There are a number of ways to do this, but we'll use the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance). The big idea is that we want to measure how often two senators vote in the same way. 

Calculations using Hamming distance are implemented in the scipy spatial distance module and we can easily convert a DataFrame of observations into a distance matrix using the [`pdist` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html#scipy.spatial.distance.pdist).

The only problem is that `pdist` wants the input array to be numeric, and we currently have strings. However, there are tools available to help with this. The preprocessing module of scikit-learn contains a [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) class which can convert a column of text into numeric labels.

In order to be able to do this, we need to take our data and convert it so that we have a single column that contains all of the votes. This is similar to pivot_wider from the tidyverse. The _pandas_ equivalent is [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html).

**Step 1:** Use the `melt` method to convert the votes dataframe into longform with three columns:
1. senator, which contains the name of the senator
2. variable, which contains the vote number
3. value, which contains the vote

In [ ]:
votes = votes.melt(id_vars = 'senator')
votes.head()

**Step 2:** Create and fit a LabelEncoder on the value column of the votes DataFrame.

In [ ]:
le = LabelEncoder().fit(votes['value'])

**Step 3:** Use the `transform` method to convert the value column to a numeric value.

In [ ]:
votes['value'] = le.transform(votes['value'])

**Step 4:** Convert the dataframe back to wide format using the [`pivot`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) method.

In [ ]:
votes = votes.pivot(index = 'senator', columns = 'variable', values = 'value')

**Question:** Which type of vote does the number 3 correspond to? (Hint: look at the methods of your LabelEncoder object).

**Step 5:** Create a matrix called `distances` which contains the Hamming distance between observations.

In [ ]:
distances = pdist(votes, metric = 'hamming')

Now, we can use the linkage function to build a dendogram. First, we'll try out using complete linkages.

To make it easier to read, we can export to a png file.

In [ ]:
mergings = linkage(distances, method='complete')

plt.figure(figsize = (12,4))
dendrogram(mergings,
           labels = votes.index,
           leaf_rotation = 90,
           leaf_font_size = 6);

plt.tight_layout()
plt.savefig('images/dendogram_complete.png', transparent=False, facecolor='white', dpi = 150)

**Question:** The top horizontal line occurs at a y-value of around 0.9. How do we interpret this value?


**Question:** Notice that the Graham/Collins/Murkowski cluster merges with the rest of the Republicans around the value of 0.25 on the y-axis. How do we interpret this?

Lindsey Graham ends up being first clustered with Susan Collins and Lisa Murkowski.

**True or False:** In terms of Hamming distance, Lindsey Graham is closer to Susan Collins and Lisa Murkowski than any other Senators.

In [ ]:
votes.index.tolist().index('Graham (R-SC)')

In [ ]:
graham_distances = pd.DataFrame({'senator': votes.index, 
                                 'distance': squareform(distances)[votes.index.tolist().index('Graham (R-SC)'), :]})

In [ ]:
graham_distances.sort_values('distance').head(10)

Now, let's try it using the single linkage method.

In [ ]:
mergings = linkage(distances, method='single')

plt.figure(figsize = (12,4))
dendrogram(mergings,
           labels = votes.index,
           leaf_rotation = 90,
           leaf_font_size = 6);

plt.tight_layout()
plt.savefig('images/dendogram_single.png', transparent=False, facecolor='white', dpi = 150)

**Question:** What differences are there between the single-linkage and complete-linkage versions of the dendograms?

**Question:** Why is the y-value at which all clusters are combined so much lower when using single-linkage?

If we want to get clusters out of our dendogram, we can make use of the [AgglomerativeClustering class](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) from scikit-learn.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In order to use this, we'll need to pass in our distances and let it know that we want to use our precomputed values.

Note that pdist returns a condensed distance matrix, whereas AgglomerativeClustering expects a square distance matrix. We can convert between the two using the `squareform` function.

We can specify the number of clusters that we want.

In [ ]:
cluster = AgglomerativeClustering(affinity = 'precomputed', 
                                  linkage = 'complete',
                                  n_clusters = 2                               
                                 ).fit(squareform(distances))

Let's try out a couple of different values for the number of clusters. To evaluate the resulting clustering, we'll use [silhouette scores](https://en.wikipedia.org/wiki/Silhouette_(clustering)) which are implemented in the scikit learn metric module: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html.

In [ ]:
from sklearn.metrics import silhouette_score

Fill in the argument values to compute the silhouette score.

In [ ]:
silhouette_score(
    X = squareform(distances), 
    labels = cluster.labels_, 
    metric = 'precomputed')

In [ ]:
max_clusters = 10

silhouette_scores = []

for n_clusters in range(2, max_clusters + 1):
    cluster = AgglomerativeClustering(affinity = 'precomputed', 
                                      linkage = 'complete',
                                      n_clusters = n_clusters).fit(squareform(distances))
    
    silhouette_scores.append(silhouette_score(squareform(distances), cluster.labels_, metric = 'precomputed'))

In [ ]:
plt.figure(figsize = (10,6))

plt.plot(range(2, max_clusters + 1), silhouette_scores);

Based on this, how many clusters should we use?

Finally, let's compare the above results to those of the first session of the 115th Congress, which took place in 2017.

The results are contained in file votes_2017.csv.

In [ ]:
votes = (
    pd.read_csv('data/votes_2017.csv')
    .dropna()
    .melt(id_vars = 'senator')
)
 
le = LabelEncoder().fit(votes['value'])
votes['value'] = le.transform(votes['value'])

votes = votes.pivot(index = 'senator', columns = 'variable', values = 'value')

distances = pdist(votes, metric = 'hamming')

In [ ]:
mergings = linkage(distances, method='complete')

plt.figure(figsize = (12,4))
dendrogram(mergings,
           labels = votes.index,
           leaf_rotation = 90,
           leaf_font_size = 6);

plt.tight_layout()
plt.savefig('images/dendogram_complete_2017.png', transparent=False, facecolor='white', dpi = 150)

What do you notice when you compare this dendogram to the one above?

**Coding Challenge:** The above results show what happens when we compute Hamming distance, including any "Not Voting" results. This results in Mike Rounds, who has missed almost a third of his votes (he is the most absent Senator), to be quite far away from any other Senator.

Redo the calculations but when making you distance calculations, only consider votes where both Senators either voted "Yea" or "Nay".